# Interacting with Databases

Another day-to-day task you will encounter (especially as a data person) is interacting with databases. Most database systems will have a python SDK available and these tend to follow a common connection/cursor paradigm.

(In fact, this paradigm spans most programming languages so you'll be a leg up when you move on to your next one!)

First, we instantiate a connection to the database. Then we need to create a `cursor`. A cursor can then execute queries and return results. Let's look at an example

Side Note: We're using sqlite here for simplicity and portability but sqlite is a pretty neat thing you should look into more - very useful easy to use tool for mucking about with databases.

In [ ]:
import sqlite3

# Since this is an unencrypted file no real connection specs are needed. In the real world
# you'll probably need to pass in host, credentials, port, etc.

db = sqlite3.connect("/home/jovyan/files/cities.db")

In [ ]:
# Creating a cursor object
cursor = db.cursor()

# We're going to pass a simple query in as a string
query = "select * from cities limit 10"

# And now we can execute it
cursor.execute(query)

## Where are the results?

`execute()` stores the results of the query in the cursor object. We need to fetch them. Here we will cheat a little and just fetch all results at once, since we know the results are small. In reality you will more often than not want to use `fetchone()` to grab one row of results. Cursors and their results are basically iterables, so you can use any kind of looping construct to handle results a row at a time and not explode your memory.

In [ ]:
cursor.fetchall()

In [ ]:
# Note that, as with all iterators, using them consumes the iterable. However fetching a consumed
# cursor will not throw an exception. Curses!

cursor.fetchone()

## DDLs and commits

Let's take a look at DDLs real quick. The concepts are very similar but we'll dig a little into the details of transactions and commits. Let's get to it:

In [ ]:
# Let's make a new database (& file)
new_db = sqlite3.connect("/home/jovyan/files/texteditors.db")

In [ ]:
# Time for the DDL statement

create_table = """CREATE TABLE text_editors
    (name text, rating int, notes text)"""

# Let's actually take advantage of doing this in code... Here's some data:

gospel_truth = {
    "sublime": {"name": "sublime", "rating": 8, "notes": "old but trusty"},
    "atom": {"name": "atom", "rating": 8, "notes": "open source is cool"},
    "word": {"name": "word", "rating": -100, "notes": "what are doing?"},
    "emacs": {"name": "emacs", "rating": 9, "notes": "begrudging respect, emacs"},
    "vim": {"name": "vim", "rating": 10, "notes": "the one true editor"}
}

# We'll need this later...
# Note that we're using the library's server-side interpolation (the ?'s)
# This is to keep us safe from SQL injection attacks... try to avoid standard
# interpolation when programatically creating queries

insert_statement = "INSERT INTO text_editors VALUES (?, ?, ?)"

In [ ]:
# Let's parse our data into a tidy iterable using a comprehension
data_tuple = [tuple(i.values()) for i in gospel_truth.values()]

In [ ]:
# Let's get a cursor and make the table
new_cursor = new_db.cursor()
new_cursor.execute(create_table)

In [ ]:
# executemany() will execute the statement against every element in the iterable... awesome!
new_cursor.executemany(insert_statement, data_tuple)

In [ ]:
# We'll talk about this in a second. Note that we commit from the connection, not
# the cursor...
new_db.commit()

In [ ]:
# How'd we do?
new_cursor.execute("select * from text_editors where rating > 8")
new_cursor.fetchall()

### What happens if we don't commit?

Each connection is a transaction. This means that if you don't commit the database will roll back to its prior state. DDLs in most systems are autocommitted, but DMLs such as `insert` and `delete` are not... let's look at that:

In [ ]:
# let's get another connection
rollback_example_conn = sqlite3.connect("/home/jovyan/files/texteditors.db")
rollback_cur = rollback_example_conn.cursor()

rollback_cur.execute("INSERT INTO text_editors VALUES ('pycharm', 5, 'ides make you weak')")

In [ ]:
# force closing the connection without committing
# We'll come back and uncomment this commit command in a sec...

# rollback_example_conn.commit()
rollback_example_conn.close()

In [ ]:
new_cursor.execute("select * from text_editors")
new_cursor.fetchall()

Let's go back, uncomment the commit statement, and try the whole thing again... 


## SQLAlchemy

SQLAlchemy is a popular library for interacting with databases that takes a different, more object-oriented approach to interacting with databases. The standard approach we covered above is more ubiquitous but you should probably [look into SQLAlchemy](https://www.sqlalchemy.org/) at some point in the future.

## A SQL Injection Interlude
![bobby_tables](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)